# Big data? 🤗 Datasets to the rescue!

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
%%capture
!pip install datasets evaluate transformers[sentencepiece]

Also, log into Hugging face.

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Nowadays it is not uncommon to find yourself working with <font color='blue'>multi-gigabyte datasets</font>, especially if you're planning to <font color='blue'>pretrain a transformer</font> like <font color='blue'>BERT</font> or <font color='blue'>GPT-2</font> from scratch. In these cases, even <font color='blue'>loading the data</font> can be a <font color='blue'>challenge</font>. For example, the <font color='blue'>WebText corpus</font> used to pretrain GPT-2 consists of over <font color='blue'>8 million documents</font> and <font color='blue'>40 GB of text</font> -- loading this into your laptop's RAM is likely to give it a heart attack!

Fortunately, 🤗 Datasets has been designed to overcome these limitations. It frees you from <font color='blue'>memory management</font> problems by treating <font color='blue'>datasets as _memory-mapped_ files</font>, and from <font color='blue'>hard drive limits</font> by _streaming_ the entries in a corpus.

<Youtube id="JwISwTCPPWo"/>

In this section we'll explore these features of 🤗 Datasets with a huge 825 GB corpus known as [the Pile](https://pile.eleuther.ai). Let's get started!

## What is the Pile?

The Pile is an <font color='blue'>English text corpus</font> that was created by [EleutherAI](https://www.eleuther.ai) for training large-scale language models. It includes a <font color='blue'>diverse range of datasets</font>, spanning <font color='blue'>scientific articles</font>, GitHub <font color='blue'>code repositories</font>, and filtered <font color='blue'>web text</font>. The training corpus is available in [14 GB chunks](https://the-eye.eu/public/AI/pile/), and you can also download several of the [individual components](https://the-eye.eu/public/AI/pile_preliminary_components/). Let's start by taking a look at the <font color='blue'>PubMed Abstracts dataset</font>, which is a corpus of <font color='blue'>abstracts</font> from <font color='blue'>15 million biomedical publications</font> on [PubMed](https://pubmed.ncbi.nlm.nih.gov/). The dataset is in [JSON Lines format](https://jsonlines.org) and is compressed using the `zstandard` library, so first we need to install that:

In [3]:
%%capture
!pip install zstandard

Next, we can <font color='blue'>load the dataset</font> using the method for remote files that we learned in [section 2](https://huggingface.co/course/chapter5/2):

In [12]:
from datasets import load_dataset, DownloadConfig

data_files = "https://huggingface.co/datasets/casinca/PUBMED_title_abstracts_2019_baseline/resolve/main/PUBMED_title_abstracts_2019_baseline.jsonl.zst"
pubmed_dataset = load_dataset(
    "json",
    data_files=data_files,
    split="train",
    download_config=DownloadConfig(delete_extracted=True),  # optional argument
)

(…)_title_abstracts_2019_baseline.jsonl.zst:   0%|          | 0.00/6.86G [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/42 [00:00<?, ?it/s]

In [13]:
pubmed_dataset.shape

(15518009, 2)

We can see that there are <font color='blue'>15,518,009 rows and 2 columns</font> in our dataset -- that's a lot!

<Tip>

✎ By default, 🤗 Datasets will decompress the files needed to load a dataset. If you want to preserve hard drive space, you can pass `DownloadConfig(delete_extracted=True)` to the `download_config` argument of `load_dataset()`. See the [documentation](https://huggingface.co/docs/datasets/package_reference/builder_classes#datasets.DownloadConfig) for more details.

</Tip>

Let's inspect the contents of the first example:


In [5]:
pubmed_dataset[0]

{'meta': {'pmid': 11409574, 'language': 'eng'},
 'text': 'Epidemiology of hypoxaemia in children with acute lower respiratory infection.\nTo determine the prevalence of hypoxaemia in children aged under 5 years suffering acute lower respiratory infections (ALRI), the risk factors for hypoxaemia in children under 5 years of age with ALRI, and the association of hypoxaemia with an increased risk of dying in children of the same age. Systematic review of the published literature. Out-patient clinics, emergency departments and hospitalisation wards in 23 health centres from 10 countries. Cohort studies reporting the frequency of hypoxaemia in children under 5 years of age with ALRI, and the association between hypoxaemia and the risk of dying. Prevalence of hypoxaemia measured in children with ARI and relative risks for the association between the severity of illness and the frequency of hypoxaemia, and between hypoxaemia and the risk of dying. Seventeen published studies were found that i

Okay, this looks like the <font color='blue'>abstract</font> from a <font color='blue'>medical article</font>. Now let's see how much RAM we've used to load the dataset!

## The magic of memory mapping

A simple way to <font color='blue'>measure memory usage</font> in Python is with the [`psutil`](https://psutil.readthedocs.io/en/latest/) library, which can be installed with `pip` as follows:

In [6]:
%%capture
!pip install psutil

It provides a `Process` class that allows us to check the memory usage of the current process as follows

In [7]:
import psutil

# Process.memory_info is expressed in bytes, so convert to megabytes
print(f"RAM used: {psutil.Process().memory_info().rss / (1024 * 1024):.2f} MB")

RAM used: 878.27 MB


Here the `rss` attribute refers to the <font color='blue'>resident set size</font>, which is the <font color='blue'>fraction of memory</font> that a process occupies in <font color='blue'>RAM</font>. This measurement also includes the memory used by the Python interpreter and the libraries we've loaded, so the actual amount of memory used to load the dataset is a bit smaller. For comparison, let's see how <font color='blue'>large the dataset</font> is on <font color='blue'>disk</font>, using the `dataset_size` attribute. Since the result is expressed in bytes like before, we need to manually convert it to gigabytes:

In [8]:
print(f"Number of files in dataset : {pubmed_dataset.dataset_size}")
size_gb = pubmed_dataset.dataset_size / (1024**3)
print(f"Dataset size (cache file) : {size_gb:.2f} GB")

Number of files in dataset : 20978892555
Dataset size (cache file) : 19.54 GB


Nice -- despite it being almost 20 GB large, we're able to load and access the dataset with much less RAM!

<Tip>

✏️ **Try it out!** Pick one of the [subsets](https://the-eye.eu/public/AI/pile_preliminary_components/) from the <font color='blue'>Pile</font> that is <font color='blue'>larger</font> than your <font color='blue'>laptop or desktop's RAM</font>, load it with 🤗 Datasets, and measure the amount of RAM used. Note that to get an accurate measurement, you'll want to do this in a new process. You can find the decompressed sizes of each subset in Table 1 of [the Pile paper](https://arxiv.org/abs/2101.00027).

</Tip>


In [9]:
# Load the C4 dataset
dataset_name = "allenai/c4"
config_name = "en"

# Load the dataset
c4_dataset = load_dataset(
    dataset_name,
    config_name,
    split="train",
    streaming=True  # Streaming to handle large dataset efficiently
)

README.md:   0%|          | 0.00/41.1k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

In [10]:
# Measure RAM usage after initializing the dataset
print(f"RAM used before accessing elements: {psutil.Process().memory_info().rss / (1024 * 1024):.2f} MB")

RAM used before accessing elements: 1346.95 MB


In [11]:
# Access a few elements to trigger the data loading
sample_elements = [next(iter(c4_dataset)) for _ in range(5)]

# Print each element
for i, element in enumerate(sample_elements):
    print(f"Sample element {i + 1}:{element}\n")

Sample element 1:{'text': 'Beginners BBQ Class Taking Place in Missoula!\nDo you want to get better at making delicious BBQ? You will have the opportunity, put this on your calendar now. Thursday, September 22nd join World Class BBQ Champion, Tony Balay from Lonestar Smoke Rangers. He will be teaching a beginner level class for everyone who wants to get better with their culinary skills.\nHe will teach you everything you need to know to compete in a KCBS BBQ competition, including techniques, recipes, timelines, meat selection and trimming, plus smoker and fire information.\nThe cost to be in the class is $35 per person, and for spectators it is free. Included in the cost will be either a t-shirt or apron and you will be tasting samples of each meat that is prepared.', 'timestamp': '2019-04-25 12:57:54', 'url': 'https://klyq.com/beginners-bbq-class-taking-place-in-missoula/'}

Sample element 2:{'text': 'Beginners BBQ Class Taking Place in Missoula!\nDo you want to get better at making 

In [12]:
# Estimate the dataset size if feasible
if hasattr(c4_dataset, 'dataset_size'):
    size_gb = c4_dataset.dataset_size / (1024**3)
    print(f"Dataset size (cache file): {size_gb:.2f} GB")
else:
    print("Dataset size attribute not available.")

Dataset size (cache file): 1543.37 GB


If you're familiar with Pandas, this result might come as a surprise because of Wes Kinney's famous [rule of thumb](https://wesmckinney.com/blog/apache-arrow-pandas-internals/) that you typically need <font color='blue'>5 to 10 times</font> as much <font color='blue'>RAM</font> as the <font color='blue'>size of your dataset</font>. So how does 🤗 Datasets solve this memory management problem? 🤗 Datasets treats each dataset as a [memory-mapped file](https://en.wikipedia.org/wiki/Memory-mapped_file), which provides a <font color='blue'>mapping</font> between <font color='blue'>RAM</font> and <font color='blue'>filesystem storage</font> that allows the library to access and operate on elements of the dataset without needing to fully load it into memory.

<font color='blue'>Memory-mapped files</font> can also be <font color='blue'>shared across multiple processes</font>, which enables methods like `Dataset.map()` to be parallelized without needing to move or copy the dataset. Under the hood, these capabilities are all realized by the [Apache Arrow](https://arrow.apache.org) memory format and [`pyarrow`](https://arrow.apache.org/docs/python/index.html) library, which make the data loading and processing lightning fast. (For more details about Apache Arrow and comparisons to Pandas, check out [Dejan Simic's blog post](https://towardsdatascience.com/apache-arrow-read-dataframe-with-zero-memory-69634092b1a).) To see this in action, let's run a little speed test by iterating over all the elements in the PubMed Abstracts dataset:

In [13]:
import timeit

code_snippet = """batch_size = 1000

for idx in range(0, len(pubmed_dataset), batch_size):
    _ = pubmed_dataset[idx:idx + batch_size]
"""

time = timeit.timeit(stmt=code_snippet, number=1, globals=globals())
print(
    f"Iterated over {len(pubmed_dataset)} examples (about {size_gb:.1f} GB) in "
    f"{time:.1f}s, i.e. {size_gb/time:.3f} GB/s"
)

Iterated over 15518009 examples (about 1543.4 GB) in 243.9s, i.e. 6.327 GB/s


Here we've used Python's `timeit` module to measure the execution time taken by `code_snippet`. You'll typically be able to iterate over a dataset at speed of a <font color='blue'>few tenths of a GB/s to several GB/s</font>. This works great for the vast majority of applications, but sometimes you'll have to work with a dataset that is too large to even store on your laptop's hard drive. For example, if we tried to download the Pile in its entirety, we'd need 825 GB of free disk space! To handle these cases, 🤗 Datasets provides a <font color='blue'>streaming feature</font> that allows us to <font color='blue'>download and access elements on the fly</font>, without needing to download the whole dataset. Let's take a look at how this works.

<Tip>

💡 In Jupyter notebooks you can also time cells using the [`%%timeit` magic function](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit).

</Tip>

## Streaming datasets

To <font color='blue'>enable dataset streaming</font> you just need to pass the `streaming=True` argument to the `load_dataset()` function. For example, let's load the PubMed Abstracts dataset again, but in streaming mode:

In [14]:
pubmed_dataset_streamed = load_dataset(
    "json", data_files=data_files, split="train", streaming=True
)

Instead of the familiar `Dataset` that we've encountered elsewhere in this chapter, the <font color='blue'>object returned</font> with `streaming=True` is an <font color='blue'>IterableDataset</font>. As the name suggests, to access the elements of an `IterableDataset` we need to iterate over it. We can access the first element of our streamed dataset as follows:


In [15]:
next(iter(pubmed_dataset_streamed))

{'meta': {'pmid': 11409574, 'language': 'eng'},
 'text': 'Epidemiology of hypoxaemia in children with acute lower respiratory infection.\nTo determine the prevalence of hypoxaemia in children aged under 5 years suffering acute lower respiratory infections (ALRI), the risk factors for hypoxaemia in children under 5 years of age with ALRI, and the association of hypoxaemia with an increased risk of dying in children of the same age. Systematic review of the published literature. Out-patient clinics, emergency departments and hospitalisation wards in 23 health centres from 10 countries. Cohort studies reporting the frequency of hypoxaemia in children under 5 years of age with ALRI, and the association between hypoxaemia and the risk of dying. Prevalence of hypoxaemia measured in children with ARI and relative risks for the association between the severity of illness and the frequency of hypoxaemia, and between hypoxaemia and the risk of dying. Seventeen published studies were found that i

The elements from a streamed dataset can be <font color='blue'>processed on the fly</font> using <font color='blue'>`IterableDataset.map()`</font>, which is <font color='blue'>useful during training</font> if you need to <font color='blue'>tokenize the inputs</font>. The process is exactly the same as the one we used to tokenize our dataset in [Chapter 3](https://huggingface.co/course/chapter3), with the only difference being that outputs are returned one by one:

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenized_dataset = pubmed_dataset_streamed.map(lambda x: tokenizer(x["text"]))
next(iter(tokenized_dataset))

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

{'meta': {'pmid': 11409574, 'language': 'eng'},
 'text': 'Epidemiology of hypoxaemia in children with acute lower respiratory infection.\nTo determine the prevalence of hypoxaemia in children aged under 5 years suffering acute lower respiratory infections (ALRI), the risk factors for hypoxaemia in children under 5 years of age with ALRI, and the association of hypoxaemia with an increased risk of dying in children of the same age. Systematic review of the published literature. Out-patient clinics, emergency departments and hospitalisation wards in 23 health centres from 10 countries. Cohort studies reporting the frequency of hypoxaemia in children under 5 years of age with ALRI, and the association between hypoxaemia and the risk of dying. Prevalence of hypoxaemia measured in children with ARI and relative risks for the association between the severity of illness and the frequency of hypoxaemia, and between hypoxaemia and the risk of dying. Seventeen published studies were found that i

<Tip>

💡 To <font color='blue'>speed up tokenization</font> with <font color='blue'>streaming</font> you can pass <font color='blue'>`batched=True`</font>, as we saw in the last section. It will process the examples batch by batch; the <font color='blue'>default batch size is 1,000</font> and can be specified with the `batch_size` argument.

</Tip>

You can also <font color='blue'>shuffle a streamed dataset</font> using `IterableDataset.shuffle()`, but unlike `Dataset.shuffle()` this only shuffles the elements in a predefined `buffer_size`:


In [17]:
shuffled_dataset = pubmed_dataset_streamed.shuffle(buffer_size=10_000, seed=42)
next(iter(shuffled_dataset))

{'meta': {'pmid': 11410799, 'language': 'eng'},
 'text': 'Randomized study of dose or schedule modification of granulocyte colony-stimulating factor in platinum-based chemotherapy for elderly patients with lung cancer.\nIt is generally believed that elderly patients are less able to tolerate aggressive cancer chemotherapy than their younger counterparts. Bone marrow cellularity diminishes with age and elderly patients may have decreased tolerance to myelosuppressive agents. Between November 1995 and October 1999, 68 chemotherapy-naive elderly (70 or more years old) patients with histologically or cytologically proven lung cancer who were to receive platinum-based chemotherapy were enrolled in this study. All patients had adequate cardiac, hematological, liver and renal function to receive chemotherapy. Patients were randomized into 3 groups. Patients in groups 1 and 2 received 2 microg/kg and 4 microg/kg granulocyte colony-stimulating factor (G-CSF, lenograstim), respectively, when gra

In this example, we selected a <font color='blue'>random example from the first 10,000 examples</font> in the buffer. <font color='blue'>Once</font> an <font color='blue'>example</font> is <font color='blue'>accessed</font>, its <font color='blue'>spot in the buffer</font> is <font color='blue'>filled</font> with the <font color='blue'>next example</font> in the corpus (i.e., the 10,001st example in the case above). You can also select elements from a streamed dataset using the `IterableDataset.take()` and `IterableDataset.skip()` functions, which act in a similar way to `Dataset.select()`. For example, to <font color='blue'>select the first 5 examples</font> in the PubMed Abstracts dataset we can do the following:

In [18]:
dataset_head = pubmed_dataset_streamed.take(5)
list(dataset_head)

[{'meta': {'pmid': 11409574, 'language': 'eng'},
  'text': 'Epidemiology of hypoxaemia in children with acute lower respiratory infection.\nTo determine the prevalence of hypoxaemia in children aged under 5 years suffering acute lower respiratory infections (ALRI), the risk factors for hypoxaemia in children under 5 years of age with ALRI, and the association of hypoxaemia with an increased risk of dying in children of the same age. Systematic review of the published literature. Out-patient clinics, emergency departments and hospitalisation wards in 23 health centres from 10 countries. Cohort studies reporting the frequency of hypoxaemia in children under 5 years of age with ALRI, and the association between hypoxaemia and the risk of dying. Prevalence of hypoxaemia measured in children with ARI and relative risks for the association between the severity of illness and the frequency of hypoxaemia, and between hypoxaemia and the risk of dying. Seventeen published studies were found that

Similarly, you can use the `IterableDataset.skip()` function to <font color='blue'>create training and validation splits</font> from a shuffled dataset as follows:

In [19]:
# Skip the first 1,000 examples and include the rest in the training set
train_dataset = shuffled_dataset.skip(1000)
# Take the first 1,000 examples for the validation set
validation_dataset = shuffled_dataset.take(1000)

Let's round out our exploration of dataset <font color='blue'>streaming</font> with a <font color='blue'>common application</font>: <font color='blue'>combining multiple datasets together</font> to create a <font color='blue'>single corpus</font>. 🤗 Datasets provides an `interleave_datasets()` function that converts a list of `IterableDataset` objects into a single `IterableDataset`, where the elements of the new dataset are obtained by <font color='blue'>alternating among the source examples</font>. This function is especially useful when you're trying to combine large datasets, so as an example let's stream the FreeLaw subset of the Pile, which is a 51 GB dataset of legal opinions from US courts:

In [6]:
# The link to the Law dataset is invalid!
from datasets import load_dataset, DownloadConfig

# Load the BookCorpus dataset from Hugging Face
dataset_name = "bookcorpus"

# Load the dataset with streaming
bookcorpus_dataset_streamed = load_dataset(
    dataset_name,
    split="train",
    streaming=True  # Streaming to handle large dataset efficiently
)

In [15]:
# Estimate the dataset size if feasible
if hasattr(bookcorpus_dataset_streamed, 'dataset_size'):
    size_gb = bookcorpus_dataset_streamed.dataset_size / (1024**3)
    print(f"Dataset size (cache file): {size_gb:.2f} GB")
else:
    print("Dataset size attribute not available.")

Dataset size (cache file): 4.52 GB


This dataset is large enough to stress the RAM of most laptops, yet we've been able to load and access it without breaking a sweat! Let's now combine the examples from the <font color='blue'>FreeLaw</font> and <font color='blue'>PubMed Abstracts datasets</font> with the `interleave_datasets()` function:

In [16]:
from itertools import islice
from datasets import interleave_datasets
import json

combined_dataset = interleave_datasets([pubmed_dataset_streamed, bookcorpus_dataset_streamed])
sampled_data = list(islice(combined_dataset, 2))

# Print the dataset samples
for i, entry in enumerate(sampled_data):
    print(f"Sample {i+1}:")
    print(json.dumps(entry, indent=4, ensure_ascii=False))
    print("-" * 80)  # Separator for readability

Sample 1:
{
    "meta": {
        "pmid": 11409574,
        "language": "eng"
    },
    "text": "Epidemiology of hypoxaemia in children with acute lower respiratory infection.\nTo determine the prevalence of hypoxaemia in children aged under 5 years suffering acute lower respiratory infections (ALRI), the risk factors for hypoxaemia in children under 5 years of age with ALRI, and the association of hypoxaemia with an increased risk of dying in children of the same age. Systematic review of the published literature. Out-patient clinics, emergency departments and hospitalisation wards in 23 health centres from 10 countries. Cohort studies reporting the frequency of hypoxaemia in children under 5 years of age with ALRI, and the association between hypoxaemia and the risk of dying. Prevalence of hypoxaemia measured in children with ARI and relative risks for the association between the severity of illness and the frequency of hypoxaemia, and between hypoxaemia and the risk of dying. Seven

Here we've used the `islice()` function from Python's `itertools` module to select the <font color='blue'>first two examples</font> from the <font color='blue'>combined dataset</font>, and we can see that they match the first examples from each of the two source datasets.

Finally, if you want to stream the <font color='blue'>Pile in its 825 GB entirety</font>, you can grab all the prepared files as follows:

In [29]:
# Loading the Pile in its entirety definitely didn't work

# Load the English Wikipedia dataset
wikipedia_dataset = load_dataset("wikipedia", "20220301.en", split="train", streaming=True)

README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

wikipedia.py:   0%|          | 0.00/36.7k [00:00<?, ?B/s]

The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [30]:
# Access a few elements
sample_elements = [next(iter(wikipedia_dataset)) for _ in range(5)]
for i, element in enumerate(sample_elements):
    print(f"Sample element {i + 1}:{element}\n")

Sample element 1:{'id': '12', 'url': 'https://en.wikipedia.org/wiki/Anarchism', 'title': 'Anarchism', 'text': 'Anarchism is a political philosophy and movement that is sceptical of authority and rejects all involuntary, coercive forms of hierarchy. Anarchism calls for the abolition of the state, which it holds to be unnecessary, undesirable, and harmful. As a historically left-wing movement, placed on the farthest left of the political spectrum, it is usually described alongside communalism and libertarian Marxism as the libertarian wing (libertarian socialism) of the socialist movement, and has a strong historical association with anti-capitalism and socialism.\n\nHumans lived in societies without formal hierarchies long before the establishment of formal states, realms, or empires. With the rise of organised hierarchical bodies, scepticism toward authority also rose. Although traces of anarchist thought are found throughout history, modern anarchism emerged from the Enlightenment. Du

In [31]:
# Estimate the dataset size if feasible
if hasattr(wikipedia_dataset, 'dataset_size'):
    size_gb = wikipedia_dataset.dataset_size / (1024**3)
    print(f"Dataset size (cache file): {size_gb:.2f} GB")
else:
    print("Dataset size attribute not available.")

Dataset size (cache file): 18.88 GB


<Tip>

✏️ **Try it out!** Use one of the large Common Crawl corpora like [`mc4`](https://huggingface.co/datasets/mc4) or [`oscar`](https://huggingface.co/datasets/oscar) to create a <font color='blue'>streaming multilingual dataset</font> that represents the spoken proportions of languages in a country of your choice. For example, the four national languages in Switzerland are German, French, Italian, and Romansh, so you could try creating a Swiss corpus by sampling the Oscar subsets according to their spoken proportion.

</Tip>

In [32]:
# Exercise

# Define language proportions for Switzerland
language_proportions = {
    "de": 0.635,  # German
    "fr": 0.225,  # French
    "it": 0.081,  # Italian
    "rm": 0.005   # Romansh
}

# Load OSCAR dataset subsets for the specified languages
languages = list(language_proportions.keys())
oscar_datasets = {lang: load_dataset("oscar", f"unshuffled_deduplicated_{lang}", split="train", streaming=True) for lang in languages}

README.md:   0%|          | 0.00/303k [00:00<?, ?B/s]

oscar.py:   0%|          | 0.00/14.8k [00:00<?, ?B/s]

The repository for oscar contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/oscar.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [33]:
def sample_elements(datasets, proportions, num_samples=100):
    """
    Sample elements from the given datasets according to the given proportions.
    """
    sampled_elements = []
    total_samples = 0

    while total_samples < num_samples:
        for lang, prop in proportions.items():
            num_lang_samples = int(num_samples * prop)
            lang_dataset = datasets[lang]

            try:
                for _ in range(num_lang_samples):
                    sample = next(iter(lang_dataset))
                    # Check if the sample is unique before adding
                    if sample not in sampled_elements:
                        sampled_elements.append((lang, sample))
            except StopIteration:
                continue  # Handle end of dataset

            total_samples += num_lang_samples

            if total_samples >= num_samples:
                break

    return sampled_elements

# Sample elements according to proportions
sampled_elements = sample_elements(oscar_datasets, language_proportions, num_samples=1000)

In [34]:
# Print some of the sampled elements
for i, (lang, element) in enumerate(sampled_elements[:5]):
    print(f"Sample {i + 1} (Language: {lang}):{element}\n")

Sample 1 (Language: de):{'id': 0, 'text': 'Dosierförderbänder Getriebe Entwässerungssiebmaschine USE 1400 x 3500 mm Eimerkettenbagger Entstaubungsanlage'}

Sample 2 (Language: de):{'id': 0, 'text': 'Dosierförderbänder Getriebe Entwässerungssiebmaschine USE 1400 x 3500 mm Eimerkettenbagger Entstaubungsanlage'}

Sample 3 (Language: de):{'id': 0, 'text': 'Dosierförderbänder Getriebe Entwässerungssiebmaschine USE 1400 x 3500 mm Eimerkettenbagger Entstaubungsanlage'}

Sample 4 (Language: de):{'id': 0, 'text': 'Dosierförderbänder Getriebe Entwässerungssiebmaschine USE 1400 x 3500 mm Eimerkettenbagger Entstaubungsanlage'}

Sample 5 (Language: de):{'id': 0, 'text': 'Dosierförderbänder Getriebe Entwässerungssiebmaschine USE 1400 x 3500 mm Eimerkettenbagger Entstaubungsanlage'}



In [35]:
# Measure RAM usage
print(f"RAM used: {psutil.Process().memory_info().rss / (1024 * 1024):.2f} MB")

# Estimate the dataset size if feasible
if hasattr(wikipedia_dataset, 'dataset_size'):
    size_gb = wikipedia_dataset.dataset_size / (1024**3)
    print(f"Dataset size (cache file): {size_gb:.2f} GB")
else:
    print("Dataset size attribute not available.")

RAM used: 1868.59 MB
Dataset size (cache file): 18.88 GB


You now have all the tools you need to <font color='blue'>load and process datasets of all shapes and sizes</font> -- but unless you're exceptionally lucky, there will come a point in your NLP journey where you'll have to actually create a dataset to solve the problem at hand. That's the topic of the next section!